In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from os.path import exists
import json
from collections import defaultdict


In [ ]:
labelMap = pd.read_csv('./newLabelMappingModified.csv', header=None, engine='python')
hasMap = labelMap[~labelMap[1].isnull()]
map1 = {k:v for k, v in zip(hasMap[0], hasMap[1])}
if exists('./LabelInverseMapping.json'):
   with open('./LabelInverseMapping.json', 'r') as f:
      map2 = json.load(f)
else:
   map2 = defaultdict(lambda: [])
   for k, v in zip(hasMap[0], hasMap[1]):
      map2[v].append(k)
   for v in labelMap[labelMap[1].isnull()][0]:
      map2[v]
   with open('./LabelInverseMapping.json', 'w') as f:
      json.dump(map2,f,indent=3)

In [ ]:
subjectMap = pd.read_csv('./label_categorized_modified.csv', header=None, engine='python')
subjectMap[1] = subjectMap[1].apply(eval)
map3 = {k:v for k,v in zip(subjectMap[0], subjectMap[1])}
def mapSubjects(subs):
   res = set()
   for s in subs:
      res.update(map3[s])
   for l in list(res):
      if l in map1:
         res.add(map1[l])
   return list(res)
# map3

In [8]:
data = pd.read_csv('./english_covid_articles.tsv', delimiter='\t', usecols=['subject', 'abstract', 'title'])
data['title'] = data['title'].apply(lambda x: eval(x[1:-1]))
parse = lambda s: eval(s) if type(s)==str else s
data['subject'] = data['subject'].apply(parse)
parse = lambda t: BeautifulSoup(t, 'html.parser').text if type(t)==str else t
data['abstract'] = data['abstract'].apply(parse)

parse = lambda s: mapSubjects(s) if type(s)==list else s
data['label'] = data['subject'].apply(parse)

data.columns, data.shape

(Index(['abstract', 'subject', 'title'], dtype='object'), (558643, 3))

In [11]:
data.to_csv('./parsedAbstract.csv', index=False)

In [14]:
labeled = data[~(data['subject'].isnull())]

In [19]:
labeled.to_csv('abstractLabeled.csv', index=False)

In [24]:
labeled = pd.read_csv('./abstractLabeled.csv', na_filter=False)